In [1]:
from src.models.Encodec import Encodec
encodec = Encodec()


/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [14]:
from audiocraft.modules.codebooks_patterns import DelayedPatternProvider

pattern_provider = DelayedPatternProvider(4)

import torch
test = torch.randn(1,24000*20)

test2 = {"wav" : torch.randn(2,1,24000*20),
        "original_lens" : torch.Tensor([[24000*13],[24000*15]]).int()}

codes = encodec(test)


In [15]:
from src.models.model import MuMRVQ
from src.models.Transformer.Encoder import VanillaEncoder
from src.models.Transformer.Decoder import VanillaDecoder
model = MuMRVQ(VanillaEncoder(layers=2), VanillaDecoder(layers=2) ,encodec, debug = False, pattern="none", mask_before=True)

In [16]:
out_ = model(test2)

tensor(975, dtype=torch.int32)
tensor(37, dtype=torch.int32)
codes: torch.Size([2, 4, 1500])
padding_mask: torch.Size([2, 1501])
codes: torch.Size([2, 4, 1024])
padding_mask: torch.Size([2, 1024])
masking before with masking proba : 0.5
============== codes_masking ==============
torch.Size([2, 4, 1024])
4126 tokens were masked with random masking
shape coming out of encoder: ============
torch.Size([2, 1025, 960])
========= retained idx shape ============
torch.Size([2, 1024])
=========== Masked without embeddings shape ========
torch.Size([2, 1024, 960])
========= all_masked.shape ==========
torch.Size([2, 1024, 960])
========= around masked index ========
tensor([[1.4359e-10, 0.0000e+00, 5.4711e+34],
        [2.8188e-03, 8.0882e-03, 1.5032e-02]])
tensor([[-0.0045,  0.0112, -0.0281],
        [-0.0024, -0.0194, -0.0073]])
class token =============
torch.Size([2, 1, 960])
========= All outputs for the encoder========
-------- masked output with class token --------
torch.Size([2, 1025,

In [11]:
out_['logits'].shape

torch.Size([2, 1027, 4, 1025])

In [12]:
from torch.nn import CrossEntropyLoss

CrossEntropyLoss(ignore_index=1026)(out_['logits'][:,:,:,1:],out_['codes'])

tensor(7.2909, grad_fn=<NllLoss2DBackward0>)

In [21]:
(out_['codes'] == 1025).sum()/out_['codes'].count_nonzero()

tensor(0.2487)